In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import os
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
# file path to data and configs

#prefix = "/home/radioglaciology/thomas/radar_data/20220903-vatnajokull-day4/20220903_033000" # Vatnajokull flight
#prefix = "/home/radioglaciology/thomas/radar_data/20230315-slakbreen-day3/20230315_064228" # Slakbreen flight


#prefix = "/home/radioglaciology/thomas/radar_data/20230721-summit-day1-bench/20230721_104552"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_103641"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_103845" # 6 db higher tx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104059"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104248" # another 6 db
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104456" # 3db higher rx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104742" # 20 -> 30 mhz chirp bw
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105025" # bw back to 20, 15 mhz lo offset
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105509" # 100k pulses, 30 us transmit, 90 us rx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105921" # 100k pulses, 10 us transmit, 90 us rx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_110253" # 100k pulses, 5 us transmit, 60 us rx

#prefix = "/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_151432" # 20 us transmit
#prefix = "/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_153254" # 10 us transmit
#prefix = "/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_152404" #  5 us transmit

#prefix = "/media/radioglaciology/Extreme SSD/summit_2023_mapper/07252023/20230725_161051" # mapper

#prefix = "/Volumes/Extreme SSD/Summit2023/07232023_anna/20230723_151432"
#prefix = "/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_141923"


prefix = "/home/thomas/Documents/StanfordGrad/RadioGlaciology/eyas/radar_data/20231031_testing/20231031_203731"


# resave data as zarr for dask processing
zarr_path = pr.save_radar_data_to_zarr(prefix)

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1, "sample_idx": 10000})

In [ ]:
n_stack = 1
stacked = pr.stack(raw.chunk({'sample_idx': 1000}), n_stack).chunk({'sample_idx': -1, })

## Fast time spectrogram

In [ ]:
n_fft = 1024
n_decimate = 500

rx_samples = raw[{'pulse_idx': 1}]['radar_data']

blocks = rx_samples.coarsen(sample_idx=(n_fft), boundary='trim').construct(sample_idx=('sample_group', 'sample'))
blocks = blocks[::n_decimate, :]

spectrogram_data = xr.apply_ufunc(
    lambda x: da.fft.fftshift(da.fft.fft(x, axis=-1), axes=-1),
    blocks,
    input_core_dims=[['sample']], # The dimension operated over -- aka "don't vectorize over this"
    output_core_dims=[["frequency"]], # The output dimensions of the lambda function itself
    exclude_dims=set(("sample",)), # Dimensions to not vectorize over
    vectorize=False, # Vectorize other dimensions using a call to np.vectorize
    dask="allowed", # Allow dask to chunk and parallelize the computation
).assign_coords({'frequency': np.fft.fftshift(np.fft.fftfreq(n_fft, d=1/raw.attrs['config']['GENERATE']['sample_rate']))})


spectrogram_data

In [ ]:
spectrogram_data = spectrogram_data.compute()
spectrogram_data

In [ ]:
fig, ax = plt.subplots(facecolor='white', figsize=(10,6))
pcm = ax.pcolormesh(spectrogram_data.sample_group, spectrogram_data.frequency / 1e6, 20*np.log10(np.abs(spectrogram_data)).T, shading='nearest')#, vmin=-400, vmax=-200)
clb = fig.colorbar(pcm, ax=ax)
clb.set_label('Power [dB]')
#ax.set_xlabel('Slow Time [s]')
ax.set_xlabel("Non-dimensional time\n(as in I haven't figured out the right dimensions yet)")
ax.set_ylabel('Frequency [MHz]')
ax.set_title("Single recording")

## Slow time spectrogram

In [ ]:
start_time = 0e-6 # 3e5
input_data = stacked["radar_data"].where(stacked.fast_time.compute() >= start_time, drop=True) #.where(stacked.slow_time <= 115, drop=True).where(stacked.slow_time >= 95, drop=True)

f, t, S = scipy.signal.spectrogram(
    input_data.to_numpy().T.flatten(),
    fs=raw.attrs["config"]["GENERATE"]["sample_rate"],
    window='flattop',
    nperseg=input_data.shape[0],
    noverlap=0,
    scaling='density', mode='psd',
    return_onesided=False
)

In [ ]:
fig, ax = plt.subplots(facecolor='white', figsize=(10,6))
freq_mhz = (np.fft.fftshift(f) + raw.attrs['config']['RF0']['freq']) / 1e6
pcm = ax.pcolormesh(input_data.pulse_idx, freq_mhz, 20*np.log10(np.abs(np.fft.fftshift(S, axes=0))), shading='nearest')#, vmin=-400, vmax=-200)
clb = fig.colorbar(pcm, ax=ax)
clb.set_label('Power [dB]')
#ax.set_xlabel('Slow Time [s]')
ax.set_xlabel('Pulse Index')
ax.set_ylabel('Frequency [MHz]')
ax.set_title(f"{os.path.basename(stacked.prefix)}\nSpectrogram of received data with n_stack={n_stack},\ntime gated to >={start_time} s fast time")
#ax.set_xlim(0, 1e-5)
#ax.set_xlim(50000, 100000)

In [ ]:
#fig.savefig(f"/Users/abroome/Desktop/Summit2023/figs/{raw.basename}-spectrogram-st-stack{n_stack}_gating.png")